In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

import os
import csv

In [2]:
loc = ''#'/data/SO_predict_DATA'
db_name = 'random_train.db'#'/train_no_dup.db'

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(loc+db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None

In [4]:
con = create_connection(db_name)

In [5]:
if con is not None:
    tag_data = pd.read_sql('''SELECT Tags FROM data''',con)
    print(tag_data.shape)

(10000, 1)


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
tag_dtm = vectorizer.fit_transform(tag_data['Tags'])

In [9]:
print("Number of data points :", tag_dtm.shape[0])
print("Number of unique tags :", tag_dtm.shape[1])

Number of data points : 10000
Number of unique tags : 6205


In [10]:
tag_names = vectorizer.get_feature_names()

In [11]:
freqs = tag_dtm.sum(axis=0).A1    ##.A1 Return `self` as a flattened `ndarray`.
result = dict(zip(tag_names, freqs))

In [14]:
#Saving this dictionary to csv files.
if not os.path.isfile('tag_counts.csv'):
    with open('tag_counts.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Tags', 'Counts'])
        for key, value in result.items():
            writer.writerow([key, value])
            
tag_df = pd.read_csv("tag_counts.csv")
tag_df.head()

,Tags,Counts
0,.bash-profile,1
1,.htaccess,41
2,.htpasswd,1
3,.net,243
4,.net-1.1,2


In [15]:
tag_df_sorted = tag_df.sort_values(['Counts'], ascending=False)
tag_counts = tag_df_sorted['Counts'].values

In [16]:
tag_df_sorted.head(20)

,Tags,Counts
684,c#,745
2746,java,728
4035,php,653
2762,javascript,613
204,android,567
2839,jquery,545
688,c++,344
4300,python,320
2686,iphone,306
362,asp.net,296


In [19]:
print(tag_df_sorted['Counts'].describe())

count    6205.000000
mean        4.679291
std        24.583264
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max       745.000000
Name: Counts, dtype: float64


In [44]:
len(tag_df_sorted) - (len(tag_df_sorted[tag_df_sorted['Counts'] < 5]))

939

In [51]:
(float)(len(tag_df_sorted[tag_df_sorted['Counts'] > 4]))/len(tag_df_sorted)

0.15132957292506044

In [46]:
tag_to_remove = tag_df_sorted[tag_df_sorted['Counts'] < 5]

In [47]:
type(tag_to_remove)

pandas.core.frame.DataFrame

In [48]:
tag_to_remove.to_csv('tag_to_remove.csv',index=None)

In [49]:
tag_df_sorted[tag_df_sorted['Tags']=='tags']

,Tags,Counts
5299,tags,7
